In [2]:
import os
DATA_PATH = "../../data"
os.chdir(DATA_PATH)

In [3]:
import pandas as pd
df = pd.read_csv('./Annotated/EN/TERM_CAT_GEN_SENT_REL.csv')

In [4]:
df

,DEFINIENDUM,CATEGORY,CATEGORY_TEXT,SENTENCE,*,AFFECTS,COMPOSITION_MEDIUM,CONTAINS,DEFINED_AS,DEFINITOR,...,frame_FORM,frame_FUNCTION,frame_LOCATION,frame_MEASURES,frame_MEDIUM,frame_POSITION,frame_RESULT,frame_SIZE,frame_STUDIES,frame_TIME
0,Karst,C. Geome|D.1 Abiotic|C. Geome|C. Geome|B.2 Loss,volcanic and permafrost areas|carbonate rocks|...,Karst is a landscape created by the dissolutio...,NaN,NaN,NaN,NaN,NaN,is a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Karst,C. Geome|D.1 Abiotic|C. Geome|B. Process,continental land masses|sea level|Karst|processes,Karst is a product of processes that operate o...,NaN,NaN,NaN,NaN,NaN,is a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Karst,B.2 Loss|D.3 Property|D.1 Abiotic|D.1 Abiotic|...,solution channel ( secondary ) porosity|rock s...,Karst is terrain with distinctive hydrology an...,NaN,NaN,NaN,with distinctive hydrology and landforms,NaN,is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Karst,C. Geome|D.1 Abiotic|D.1 Abiotic|D.1 Abiotic|C...,underground water systems|soluble rocks|limest...,Karst is the term used to describe a special s...,NaN,NaN,developed on especially soluble rocks such as ...,containing caves and extensive underground wat...,NaN,is the term used to describe a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Karst conduit networks,C. Geome|C. Geome|D.1 Abiotic,Karst conduit networks|karst massif|conduits,“ Karst conduit networks ” are defined as the ...,NaN,NaN,NaN,NaN,the total length of the existing conduits with...,are defined as,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wider than,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,syngenetic karst,C. Geome|D.1 Abiotic|B. Process|B.4 Transforma...,syngenetic karst|aeolian calcarenite|evolution...,syngenetic karst - karst developed in aeolian ...,NaN,NaN,developed in aeolian calcarenite,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,when
839,tafoni,A.1 Surface landform,tafoni,tafoni - roughly hemispherical hollows weather...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,at the|in,NaN,NaN,NaN,NaN,NaN,NaN,NaN
840,through cave,A.2 Underground landform|D.1 Abiotic|D.1 Abiot...,through cave|stream course|stream|passage|cave,through cave - a cave which may be followed fr...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,which may be,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
841,tower karst,A.1 Surface landform|A.1 Surface landform|C. G...,alluvial plains|flat-floored depressions|tower...,tower karst - conekarst in which the residual ...,NaN,NaN,NaN,there my be alluvial plains between the towers...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
!pip install stanza
import stanza

     |████████████████████████████████| 574 kB 8.6 MB/s 
     |████████████████████████████████| 175 kB 55.7 MB/s 
     |████████████████████████████████| 4.0 MB 47.1 MB/s 
     |████████████████████████████████| 895 kB 45.5 MB/s 
     |████████████████████████████████| 6.6 MB 43.2 MB/s 
     |████████████████████████████████| 596 kB 59.1 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=20e7e453e11aa9fcf1bd6f663489e4015321fc3e66dbf03c83dddc64f8679b0b
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
nlp = stanza.Pipeline("en", processors='tokenize,pos,lemma,depparse')

2022-04-26 18:24:50 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-04-26 18:24:50 INFO: Use device: cpu
2022-04-26 18:24:50 INFO: Loading: tokenize
2022-04-26 18:24:50 INFO: Loading: pos
2022-04-26 18:24:50 INFO: Loading: lemma
2022-04-26 18:24:50 INFO: Loading: depparse
2022-04-26 18:24:50 INFO: Done loading processors!


# Generate data for https://github.com/Cartus/AGGCN

In [ ]:
import json
import numpy as np
from tqdm import tqdm

def process_sentence(doc):
    token = []
    stanford_pos = []
    stanford_head = []
    stanford_deprel = []
    for sentence in doc.sentences:
        for word in sentence.words:
            token.append(word.text)
            stanford_pos.append(word.xpos)
            stanford_head.append(word.head)
            stanford_deprel.append(word.deprel)
    return token, stanford_pos, stanford_head, stanford_deprel

def find_start_stop(token, token_all):
    # start = token_all.index(token[0])
    # end = token_all.index(end_token)
    possible_start = [i for i, x in enumerate(token_all) if x == token[0]]
    end_token = token[-1]
    
    end = None
    for start in possible_start:
        if token_all[start+len(token)-1] == end_token:
            end = start+len(token)-1
            break

    if end is None:
        print(token_all, token)
        print(possible_start)
        raise Exception("Couldn't find end position")

    return start, end

def check_reverse_relation(obj_start, subj_start):
    return obj_start > subj_start

def create_json_object(index, relation, token, stanford_pos, stanford_head, stanford_deprel, subj_start, subj_end, obj_start, obj_end):
    return {
        "index": index, 
        "relation": relation, 
        "token": token, 
        "stanford_pos": stanford_pos,
        "stanford_head": stanford_head,
        "stanford_deprel": stanford_deprel,
        "subj_start": subj_start,
        "subj_end": subj_end,
        "obj_start": obj_start,
        "obj_end": obj_end
    }


relations = ["AFFECTS","COMPOSITION_MEDIUM","CONTAINS","DEFINED_AS","GENUS","HAS_ATTRIBUTE","HAS_CAUSE","HAS_FORM","HAS_FUNCTION","HAS_LOCATION","HAS_POSITION","HAS_RESULT","HAS_SIZE","MEASURES","OCCURS_IN_MEDIUM","OCCURS_IN_TIME","SPECIES","STUDIES"]
relations_reverse = [r+"_rev" for r in relations]

number_of_rows =df.shape[0]
index = 0
data = []
with tqdm(total=df.shape[0]) as pbar:
    for row_ix, row in df.iterrows():
        sentence_raw = row["SENTENCE"]
        # Edge cases
        sentence_raw = sentence_raw.replace("sediment. of rock", "sediment of rock")
        sentence_raw = sentence_raw.replace("0 ° C.,", "0 ° C,")
        sentence_raw = sentence_raw.replace("(Ford, 2006; Klimchouk, 2007). ", "")

        curr_sentence = nlp(sentence_raw)
        token, stanford_pos, stanford_head, stanford_deprel = process_sentence(curr_sentence)
        obj = row["DEFINIENDUM"]
        
        # Edge cases
        if obj == "lava-cave":
            obj_token = ["lava", "-", "cave"]
        elif obj == "tecto-karstic hollow":
            obj_token = ["tecto-karstic", "hollow"]
        elif obj == "krš":
            obj_token = ["krš"]
        elif obj == "cover-collapse sinkhole":
            obj_token = ["cover", "-collapse", "sinkhole"]
        else:
            obj_token = [w.text for s in nlp(obj).sentences for w in s.words]
        obj_start, obj_end = find_start_stop(obj_token, token)

        for i, relation in enumerate(relations):
            if not pd.isna(row.at[relation]):
                subjects = row[relation].split("|")
                for subject in subjects:
                    # Edge cases
                    subject = subject.replace("e.g .", "e.g.")
                    subject = subject.replace("i.e .", "i.e.")
                    subject = subject.replace("c .", "c.")
                    subject = subject.replace(" ,",",")

                    if subject.startswith(","):
                        subject = subject[1:]
                    elif subject.endswith(" cavities"):
                        subject = subject.replace(" cavities", "")
                    elif subject == "of pure calcium carbonate thin film":
                        subject = "of pure calcium carbonate"

                    if "composite result of carbonate deposition and dissolution" in subject:
                        subj_token = ["composite", "result", "of", "carbonate", "deposition", "and", "dissolution", "cycles", ",", "which", "have", "been", "controlled", "by", "glacial", "sea", "-level", "fluctuations"]
                    elif "extend below sea-level for" in subject:
                        subj_token = ["extend", "below", "sea", "-", "level", "for", "a", "majority", "of", "their", "depth"]
                    elif "in a karst area" in subject:
                        subj_token = ["in", "a", "karst", "area"]
                    else:
                        subj_token = [w.text for s in nlp(subject).sentences for w in s.words]
                    
                    if subj_token[-1] == "etc" and obj=="pipe":
                        subj_token[-1] = "etc."

                    subj_start, subj_end = find_start_stop(subj_token, token)

                    if check_reverse_relation(obj_start, subj_start):
                        relation = relations_reverse[i]

                    curr_json = create_json_object(index, relation, token, stanford_pos, stanford_head, stanford_deprel, subj_start, subj_end, obj_start, obj_end)
                    data.append(curr_json)
                    index += 1
        pbar.update()


print(f"Number of examples: {index}")

json_data = json.dumps(data, indent=4)

with open("./Stanza/EN/karst_stanza_data.json", "w+") as outfile:
    outfile.write(json_data)

100%|██████████| 843/843 [06:32<00:00,  2.15it/s]


Number of examples: 2642


# Generate test and train set
Take only subset of relationships (GENUS, HAS_LOCATION, HAS_SIZE) 

In [28]:
import json
from sklearn.model_selection import train_test_split

with open("./Annotated/EN/karst_stanza_data.json", "r") as f:
    data = json.load(f)

relations = ["GENUS", "HAS_LOCATION", "HAS_SIZE", "GENUS_rev", "HAS_LOCATION_rev", "HAS_SIZE_rev"]
label_dict = {r:i for i, r in enumerate(relations)}

new_data = []
labels = []
for el in data:
    if el["relation"] in relations:
        new_data.append(el)
        labels.append(label_dict[el["relation"]])


X_train, X_test, y_train, y_test = train_test_split(new_data, labels, test_size=0.30, random_state=42, stratify=labels)

json_train = json.dumps(X_train, indent=4)
json_test = json.dumps(X_test, indent=4)


with open("./Stanza/EN/karst_stanza_train.json", "w+") as outfile:
    outfile.write(json_train)
    
with open("./Stanza/EN/karst_stanza_test.json", "w+") as outfile:
    outfile.write(json_test)